In [4]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [5]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [6]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [7]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [8]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -9.35436159e-04  -2.75415547e-03  -2.94843902e-03 ...,
      4.26791079e-03  -1.80280470e-03   4.15953010e-03]
   [  1.04164346e-03   5.14854865e-04   4.17316713e-03 ...,
      2.84231019e-03   2.97774200e-03   2.70415841e-04]
   [  4.85936069e-03   4.80936043e-03  -4.88157548e-03 ...,
      2.31426160e-03   4.38076606e-04   1.64524458e-03]
   ..., 
   [ -7.42720861e-04   4.26659688e-04   1.16022223e-03 ...,
     -4.18199582e-03   3.00985007e-03  -1.05423686e-03]
   [  5.85104882e-04   4.46651113e-03   2.82536639e-03 ...,
     -1.78820623e-03  -2.39032322e-03  -9.37865139e-04]
   [ -9.94892453e-04   2.22876199e-03  -3.57989390e-03 ...,
      2.64567320e-03   3.60498007e-03  -1.31180289e-03]]

  [[  2.28296994e-03  -1.08588753e-03  -4.74120590e-03 ...,
      7.30296285e-04  -1.18636011e-03  -3.55704834e-03]
   [  4.62905399e-03   4.00860870e-03  -2.53451583e-03 ...,
     -9.88519924e-04   3.63499366e-03   4.02229146e-03]
   [ -2.53732599e-03  -2.17647692e-03   1.86831488e-03 ...,
 

In [9]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x < 0] = leak
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

0.00174413544865


In [10]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x)) #Add up all cross-correlations (with ReLU)
    return new_img

In [11]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
                #print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
                #print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2]) 
                #Apply corresponding filter
                print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
[[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.23529412,  0.23529412,  1.        ,  0.23529412,  0.23529412,
         0.23529412,  0.23529412,  1.    

In [16]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis
print(pool(arr1r, 2))

[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.23529412  1.          0.23529412  1.          0.23529412  1.          1.
   1.          1.          1.          1.          1.          1.        ]
 [ 1.          0.23529412  0.23529412  1.          0.23529412  1.          1.
   1.          1.          1.          1.          1.          1.        ]
 [ 0.23529412  1.          0.23529412  1.          0.35686275  1.          1.
   1.          1.          1.          1.          1.          1.        ]
 [ 0.92941176  0.92941176  0.92941176  0.92941176  0.92941176  0.92941176
   0.92941176  0.92941176  0.92941176  0.92941176  0.92941176  0.92941176
   0.92941176]
 [ 1.          1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.        ]
 [ 1.          1.          1.          1.          1.          1.       

In [1]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#         print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error,np.sum(np.multiply(np.multiply(
                    (array(out[layer-1])),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

NameError: name 'np' is not defined

In [167]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [2]:
for x in range(epochs):
    out = forward(arr1)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
    print(filters)

NameError: name 'epochs' is not defined